In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import datetime
import random
import xarray as xr
import cartopy.crs as ccrs
import netCDF4
import matplotlib.pylab as pylab
import matplotlib.colors as mcolors
import h5py
import argparse
import pymap3d as pm
import os
import time
import math
import torch
import copy
import re
import matplotlib.patches as patches
import cartopy.feature as cfeature

from scipy import signal
from scipy import spatial
from torch.nn import Linear
from torch import Tensor
from torch.nn import MSELoss
from torch.optim import SGD, Adam, RMSprop
from torch.autograd import Variable, grad
from torch.cuda.amp import autocast
from torch.utils.data.sampler import SubsetRandomSampler,WeightedRandomSampler
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from pathlib import Path
from pyproj import Proj
from scipy.ndimage.filters import gaussian_filter
from glob import glob
from matplotlib.ticker import PercentFormatter
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes,mark_inset
from mpl_toolkits.axes_grid1 import ImageGrid

# for reproducibility
os.environ['PYTHONHASHSEED']= '123'
np.random.seed(123)

# use custom Matplotlib style
plt.style.use("./science.mplstyle")

# loading data
path_file = '../../00_data/glad-m25-vp-0.0-n4.nc'

data = xr.open_dataset(path_file)

offline = False
old = False

if offline and old:
    dep_ini, dep_inc = 11, 4
    lat_ini, lat_inc = 0, 8
    lon_ini, lon_inc = 0, 8
else:
    dep_ini, dep_inc = 11, 1
    lat_ini, lat_inc = 0, 2
    lon_ini, lon_inc = 0, 2    

# variables
vpv = data.variables['vpv'].values[dep_ini::dep_inc, lat_ini::lat_inc, lon_ini::lon_inc]
vph = data.variables['vph'].values[dep_ini::dep_inc, lat_ini::lat_inc, lon_ini::lon_inc]
longitude = data.variables['longitude'].values[lon_ini::lon_inc]
latitude = data.variables['latitude'].values[lat_ini::lat_inc]
depth = data.variables['depth'].values[dep_ini::dep_inc]

dep_dim = vpv.shape[0]
lat_dim = vpv.shape[1]
lon_dim = vpv.shape[2]

# geodetic-geocentric projection
LAT, ALT, LON = np.meshgrid(latitude, -1e3*depth, longitude)
x, y, z = pm.geodetic2ecef(LAT, LON, ALT)
_, DEP, _ = np.meshgrid(latitude, depth, longitude)

# multi-receivers options
rec_typ = 'US_array'

# type of "play ground"
experiment = "mean_receiver_velocity_all_points"

# saving parameters
num_pts = x.size
num_epo = int(2001)
num_blo = 21 #20
coo_sys = 'cartesian'
vel_sha = 'sphere'
vel_typ = 'gladm25'
num_neu = 512
lea_rat = 5e-6
act_fun = torch.nn.ELU
bat_siz = 512 #num_pts // 100
ada_wei = False
opt_fun = torch.optim.Adam
dev_typ = "cuda"
bac_vel = 10 #6 #10.5
hyp_par = (
    str(num_epo) + '_' +
    str(num_blo) + '_' +
    str(lea_rat) + '_' +
    str(dep_dim) + '_' +
    str(lat_dim) + '_' +
    str(lon_dim) + '_' +
    str(act_fun) + '_' +
    str(ada_wei) + '_' +
    str(num_pts) + '_' +
    str(bat_siz) + '_' +
    vel_sha + '_' +
    vel_typ + '_' +
    str(num_neu) + '_' +
    coo_sys + '_' +
    str(opt_fun) + '_' +
    str(bac_vel) + '_' +
    rec_typ + '_' +
    dev_typ + '_' +
    experiment
)

# # path
# model_path = "./../results/" + hyp_par
# figures_path = model_path + '/'
# checkpoints_path = figures_path + 'checkpoints' + '/'
# predictions_path = figures_path + 'predictions' + '/'

# Path(figures_path).mkdir(parents=True, exist_ok=True)
# Path(checkpoints_path).mkdir(parents=True, exist_ok=True)
# Path(predictions_path).mkdir(parents=True, exist_ok=True)

# training points
# laod all stations
ISCall = pd.read_csv('stations.csv')
ISCall = ISCall.rename(columns={"X":"LON", 'Y':'LAT'})

# laod only active stations (from ISC website) to 2021
ISCarray = ISCall[ISCall['description'].str.contains('to 2021')]
ISCarrLon = ISCarray['LON']
ISCarrLat = ISCarray['LAT']

# load US array data
USarray = pd.read_excel('_US-TA-StationList.xls')
USarrLon = USarray['LON']
USarrLat = USarray['LAT']

# concatenate the two receiver group
AllLon = np.hstack((USarrLon, ISCarrLon))
AllLat = np.hstack((USarrLat, ISCarrLat))

# model specifications
ear_rad = 6371

# source vectors
if rec_typ == 'ISC_array':
    print(rec_typ)
    lat_sou = np.array([latitude.flat[np.abs(latitude - i).argmin()] for i in ISCarrLat])
    lon_sou = np.array([longitude.flat[np.abs(longitude - i).argmin()] for i in ISCarrLon])
elif rec_typ == 'US_array':
    print(rec_typ)
    lat_sou = np.array([latitude.flat[np.abs(latitude - i).argmin()] for i in USarrLat])
    lon_sou = np.array([longitude.flat[np.abs(longitude - i).argmin()] for i in USarrLon])
dep_sou = depth.flat[np.abs(depth - 0).argmin()]

xx = (ear_rad - DEP) * np.sin(np.radians(LAT+90)) * np.cos(np.radians(180+LON))/(1e3)
yy = (ear_rad - DEP) * np.sin(np.radians(LAT+90)) * np.sin(np.radians(180+LON))/(1e3)
zz = DEP * np.cos(np.radians(LAT+90)) / (1e3)

xx_s = (ear_rad - dep_sou) * np.sin(np.radians(lat_sou+90)) * np.cos(np.radians(180+lon_sou))/(1e3)
yy_s = (ear_rad - dep_sou) * np.sin(np.radians(lat_sou+90)) * np.sin(np.radians(180+lon_sou))/(1e3)

# coordinates setup
sx, sy, sz = pm.geodetic2ecef(lat_sou, lon_sou, -1e3*dep_sou)

# rescale
X,Y,Z = x/(ear_rad*1e3), y/(ear_rad*1e3), z/(ear_rad*1e3)
sx, sy, sz = sx/(ear_rad*1e3), sy/(ear_rad*1e3), sz/(ear_rad*1e3)

US_array


In [15]:
0.7*X.size,  2**24

(15139509.7, 16777216)

In [ ]:
# finding source indices using Dask
import dask.array as da
sids = da.where((da.isclose(da.from_array(X.reshape(-1,1), chunks=X.size//100), da.from_array(sx), atol=1e-16)) & (da.isclose(da.from_array(Y.reshape(-1,1), chunks=X.size//100), da.from_array(sy), atol=1e-16)) & (da.isclose(da.from_array(Z.reshape(-1,1), chunks=X.size//100), da.from_array(sz), atol=1e-16)))[0]
sx, sy, sz = X.reshape(-1,1)[sids], Y.reshape(-1,1)[sids], Z.reshape(-1,1)[sids]
sids = sids[sids.astype(bool)].astype(int)

In [ ]:
# finding source indices using Dask
import dask.array as da

In [ ]:
da.from_array(X.reshape(-1,1), chunks=X.size//100)

In [ ]:
da.from_array(X.reshape(-1,1))